In [ ]:
from resources.iot.device import IoT_mqtt_publisher, IoT_sensor
from resources.iot.device import IoT_sensor_consumer
from IPython.core.display import display
import ipywidgets as widgets
from resources.iot.device import IoT_mqtt_publisher, IoT_sensor
from IPython.core.display import HTML 
import os
import resources.thedatasociety as thedatasociety

# Componente para simulação de um sensor

<!--
```bash
IoT_sensor(<name/id>, <grandeza física >, <unidade de medida>, <menor valor>, <maior valor possível>, <intervalo entre leituras (segundos)>)
```

Exemplo de sensor de pressão:

```python
sensor_pressao = IoT_sensor("32", "pressao", "bar", 20, 35, 5)

```
-->

Componentes `IoT_sensor` podem se conectar a componentes do tipo `IoT_mqtt_publisher` para publicar, em um tópico, mensagens referentes às leituras feitas pelo sensor. Por exemplo, o sensor do exemplo acima produziu a seguinte mensagem no tópico `sensor/32/pressao`:

```python
 {
 	"source": "sensor",
 	"name": "32",
 	"type": "mode-r2d2",
 	"body": {
 		"timestamp": "2019-08-17 17:02:15",
 		"dimension": "pressao",
 		"value": 25.533895448246717,
 		"unity": "bar"
 	}
 }
```

Simuladores podem publicar mensagens aleatórias de tempos em tempos.


# Instanciando Simuladores de Sensores

In [ ]:
sensor_1 = IoT_sensor(name="1", dimension="Temperatura",     unity="°C",  min_value=25, max_value=35,   pooling_interval=2)
sensor_2 = IoT_sensor(name="2", dimension="Umidade",         unity="%",   min_value=40, max_value=65,   pooling_interval=3)
sensor_3 = IoT_sensor(name="3", dimension="BateriaRestante", unity="%",   min_value=5,  max_value=100,  pooling_interval=4)
sensor_4 = IoT_sensor(name="4", dimension="SinalWifi",       unity="dBm", min_value=-30, max_value=-90, pooling_interval=2)

# Instanciando Componente de Publicação de Mensagens no MQTT

In [ ]:
publisher = IoT_mqtt_publisher("localhost", 1883)

### Conectando os Componentes 

In [ ]:
sensor_1.connect(publisher)
sensor_2.connect(publisher)
sensor_3.connect(publisher)
sensor_4.connect(publisher)

# Escutando o barramento

Abra um terminal e digite:

```bash

mosquitto_sub -t "sensor/#" -v

```

# Construindo um dashboard local

### Criando o consumidor_1 e seus widgets

In [ ]:
widget_1 = widgets.FloatProgress(min=0, max=40.0, bar_style='', orientation='vertical'); widget_1_label = widgets.Label()

consumer_1 = IoT_sensor_consumer("localhost",1883,"sensor/1/+")

### Criando o consumidor_2 e seus widgets

In [ ]:
widget_2 = widgets.FloatProgress(min=0, max=90.0, bar_style='warning', orientation='vertical'); widget_2_label = widgets.Label()

consumer_2 = IoT_sensor_consumer("localhost",1883,"sensor/2/+")

### Criando o consumidor_3 e seus widgets

In [ ]:
widget_3  = widgets.FloatProgress(min=0, max=100.0, bar_style='info', orientation='vertical'); widget_3_label = widgets.Label()

consumer_3 = IoT_sensor_consumer("localhost",1883,"sensor/3/+")

### Criando o consumidor_4 e seus widgets

In [ ]:
widget_4  = widgets.FloatProgress(min=-95.0, max=-30, value=-95, bar_style='success', orientation='vertical'); widget_4_label = widgets.Label()

consumer_4 = IoT_sensor_consumer("localhost",1883,"sensor/4/+")

## Organizando os componentes visualmente

In [ ]:
separator = widgets.Label(value=" ---------- ")
col_1 = widgets.VBox([widget_1, widget_1_label])
col_2 = widgets.VBox([widget_2, widget_2_label])
col_3 = widgets.VBox([widget_3, widget_3_label])
col_4 = widgets.VBox([widget_4, widget_4_label])
row_1 = widgets.HBox([separator, col_1, separator, col_2, separator, col_3, separator, col_4])
display(row_1)


### Conectando componentes visuais e seus respectivos consumidores

In [ ]:
consumer_1.connect(widget_1, widget_1_label)
consumer_2.connect(widget_2, widget_2_label)
consumer_3.connect(widget_3, widget_3_label)
consumer_4.connect(widget_4, widget_4_label)

# Consultando InfluxDB

In [ ]:
! influx --execute "show databases"

In [ ]:
! influx -execute "show measurements" -database 'exemplo_iot'

In [ ]:
! influx -execute "show measurements" -database 'exemplo_iot'

In [ ]:
! influx -execute "show series" -database 'exemplo_iot'

In [ ]:
! influx -execute "show tag keys" -database 'exemplo_iot'

In [ ]:
! influx -execute 'show tag values with key = "sensor_id"' -database 'exemplo_iot'

In [ ]:
! influx -execute 'show field keys ' -database 'exemplo_iot'

In [ ]:
! influx -execute "select value from cliente_A where medida = 'Temperatura' limit 10 " -database 'exemplo_iot'

In [ ]:
! influx -execute "select value from cliente_A where medida = 'Umidade' AND time >  now() - 10s limit 10" -database 'exemplo_iot'

In [ ]:
! influx -execute "select max(value), min(value), mean(value) from cliente_A where medida = 'Temperatura' AND time >  now() - 30s " -database 'exemplo_iot'

# Acessando o Grafana

A célula abaixo permite acessar a instância do Grafana que está rodando nesta sessão. Verifique a sua URL de acordo com o servidor utilizado.

In [ ]:
binderhub_urls= ['hub.gke.mybinder.org', 'hub-binder.mybinder.ovh', 'notebooks.gesis.org', 'hub.binder.pangeo.io']

thedatasociety.list_urls(binderhub_urls)

## Adicione um data source

Go to the setting menu (left cog on the vertical bar) and click to add a data source.

![](../resources/images/screenshot-grafana0.png)

Select InfluxDB and then set the following parameters:


- Name: influxdb
- Type: influxdb
- URL: http://localhost:8086/
- Database: telegraf
- User: telegraf
- Password: 'telegraf'

Then click `save & test`.

![](../resources/images/screenshot-grafana1.png)

## Importe um dashboard ou crie o seu

Grafana provides the repository for grafana plugins and dashboards.

- [Grafana Plugins](https://grafana.com/plugins)
- [Grafana Dashboards](https://grafana.com/dashboards)


For instance, go to [https://grafana.com/dashboards/5955](https://grafana.com/dashboards/5955) and click the 'Copy the ID to Clipboard' button.

Click on the `+` option on the left bar and select `import`. Paste the ID, select the Data Source and you will be done!